# Исследование надежности заемщиков.

### Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения 
кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает 
способность потенциального заёмщика вернуть кредит банку. 

Описание данных  
•	children — количество детей в семье  
•	days_employed — общий трудовой стаж в днях  
•	dob_years — возраст клиента в годах  
•	education — уровень образования клиента  
•	education_id — идентификатор уровня образования  
•	family_status — семейное положение  
•	family_status_id — идентификатор семейного положения  
•	gender — пол клиента  
•	income_type — тип занятости  
•	debt — имел ли задолженность по возврату кредитов  
•	total_income — ежемесячный доход  
•	purpose — цель получения кредита  


### Шаг 1. Обзор данных

Импорт библиотек

In [2]:
import pandas as pd

Чтение файлов

In [3]:
Data = pd.read_csv('/datasets/data.csv')
Data.info()
Data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Шаг 2.1 Заполнение пропусков

In [4]:
print(Data.info())
print('Количество пропущенных значений\n',Data.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
Количество пропущенных значений
 children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose        

В столбцах days_employed и total_income обнаружены пропущеннные значения

In [5]:
# Вывод доли пропущеннных значений в столбцах с пропусками
for column in Data.columns:
    if Data[column].isna().sum():
        print ('Количество пропущенных значений в столбце:', column, Data[column].isna().sum(),
               ', что составляет - ', f"{(Data[column].isna().sum()/Data.shape[0]):.2%}")

Количество пропущенных значений в столбце: days_employed 2174 , что составляет -  10.10%
Количество пропущенных значений в столбце: total_income 2174 , что составляет -  10.10%


Возможные причины появления пропусков в данных: ошибки ввода, сокрытие информации и т.д.
Доля пропущенных значений более 10%, что является существенным, переменные количественные, следовательно заполнение пропусков медианным значением наилучший вариант

In [6]:
total_income_median = Data['total_income'].median()
days_employed_median = Data['days_employed'].median()
print('Медианное значение total_income: %.2f' % total_income_median, 'рублей') 
print('Медианное значение days_employed: %.2f' % days_employed_median)

Медианное значение total_income: 145017.94 рублей
Медианное значение days_employed: -1203.37


Замена пропущенных значений в столбце total_income медианными значениями по соответствующему столбцу(в соответствии с условиями задачи). Используем медиану по уровню образования и роду деятельности

In [7]:
import math #необходима для проверки наличия NaN

Функция, которая будет вычислять медиану для конкретного набора по уровню образования и типа работы
в случае, если среднее не может быть вычислено (не найдется в основных значениях группы параметров),
возьмет среднее по одному параметру - сначала по уровню образования, потом по типу сотрудника

In [9]:
def calc_median_by_education_and_income(Data, education, income_type):
    # принимаем во внимание  оба параметра
    median = Data[(Data['education'] == education) & (Data['income_type'] == income_type)]['total_income'].median()
    # но т.к. возможна уникальная ситуация, где нет совпадения для двух параметров, предусмотрим рассчет по первому параметру 
    if math.isnan(median):
        median = Data[Data['education'] == education]['total_income'].median()
        if math.isnan(median):
            median = Data[Data['income_type'] == income_type]['total_income'].median()    
    return median

Проходим по всем уровням образования и типам сотрудников, что встречаются только для пустых значений и проставляем только для них медиану, соответствующую их группе

In [10]:
for education in Data[Data['total_income'].isnull()]['education'].unique():
    for income_type in Data[Data['total_income'].isnull()]['income_type'].unique():
        median = calc_median_by_education_and_income(Data, education, income_type)
        searchRow = (Data['education'] == education) & (Data['income_type'] == income_type) & (Data['total_income'].isnull())
        Data.loc[searchRow, 'total_income'] = Data.loc[searchRow, 'total_income'].fillna(median)

Проверяем, что в income не осталось пустых значений

In [11]:
Data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Столбец days_empoyed для целей анализа данных не нужен, а значит его можно удалить

In [12]:
# Удаляем столбец 'days_employed'
Data.drop('days_employed', axis='columns', inplace=True)
print(Data.columns) #проверка

#Data['total_income']=Data['total_income'].fillna(value=total_income_median)
#Data['days_employed']=Data['days_employed'].fillna(value=days_employed_median)
#print(Data.head(5) 

Index(['children', 'dob_years', 'education', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')


Еще раз проверяем на отсутствие пустых значений 

In [13]:
Data.isnull().sum()

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Вывод  
Несуществующим значениям дохода (total_income) поставлена медиана, но не всей выборки, а только по конкретным категориям, к которым люди относились, что позволит получить более точную информацию и снизить возможную статистическую погрешность.
Пропуски столбца общего трудового стажа в днях (days_employed) не учитываются, т.к. не несут значимой информации.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [14]:
# Проверка данных на аномалии
for row in Data: 
    print(Data[row].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее      

Для 'gender' строка с пропущенным значением всего одна. Ее можно исключить из выборки. 

In [15]:
Data = Data[Data['gender'] != 'XNA']

В таблице есть артефакты значений в графе дети, примем -1 как бездетных, а 20 как ошибку при вводе '2'

In [16]:
Data.loc[Data['children'] == -1, 'children'] = 0
Data.loc[Data['children'] == 20, 'children'] = 2
print(Data['children'].value_counts())
#Data['days_employed'] = Data['days_employed'].apply(abs) # избавляемся от отрицательных значений days_employed
Data.info() # проверка

0    14195
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 11 columns):
children            21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 2.0+ MB


Воможные причины появления аномальных данных: ошибки ввода информации первичными пользователями, избыточность данных, дефекты проектирования базы данных, к примеру вместо 1, введено -1. 

### Шаг 2.3. Изменение типов данных.

Замените вещественный тип данных в столбце total_income на целочисленный, например, с помощью метода astype()

In [17]:
Data['total_income'] = Data['total_income'].round().astype('int')
Data.head() # проверка

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.4. Удаление дубликатов.

Дубликаты можно искать двумя способами.  
Способ 1. Во вводном курсе вы уже изучали метод duplicated().В сочетании с методом sum() он возвращает количество дубликатов. 
Если выполнить метод duplicated() без подсчёта суммы, на экране будут отображены все строки. Там, где есть дубликаты, будет значение True, где дубликата нет — False.  
Способ 2. Вызвать метод value_counts(), который анализирует столбец, выбирает каждое уникальное значение и подсчитывает частоту его встречаемости в списке. Применяют метод к объекту Series. Результат его работы – список пар «значение-частота», 
отсортированные по убыванию. Все дубликаты, которые встречаются чаще других, оказываются в начале списка.
Исходя из изложенного выше выбран метод duplicated() 
Основные причины возникновения дубликатов — повторные представления, неправильное соединение данных из разных источников, ошибки пользователя при занесении информации. 

In [18]:
Data['education'] = Data['education'].str.lower()# Приведем значения в столбце Уровень образованиt клиента
# к единому регистру
Data.sort_values('education')['education'].value_counts()
print(Data.duplicated().sum()) 

54


Обнаружен 54 дубликат. Удаляем и проверяем.

In [19]:
Data = Data.drop_duplicates().reset_index()
print(Data.duplicated().sum()) # Проверка

0


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [20]:
# Словарь для education
education_dict = Data[['education_id', 'education']]

In [21]:
# Удаление дубликатов из словаря 
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict.head(10)# проверка

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [22]:
# Словарь для family_status
family_status_dict = Data[['family_status_id', 'family_status']]

In [23]:
# Удаление дубликатов из словаря
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict.head(10) # проверка

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [24]:
# Удаление столбцов education и family_status из основной таблицы
Data.pop('education')
Data.pop('family_status')
Data.info()#проверка 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21470 entries, 0 to 21469
Data columns (total 10 columns):
index               21470 non-null int64
children            21470 non-null int64
dob_years           21470 non-null int64
education_id        21470 non-null int64
family_status_id    21470 non-null int64
gender              21470 non-null object
income_type         21470 non-null object
debt                21470 non-null int64
total_income        21470 non-null int64
purpose             21470 non-null object
dtypes: int64(7), object(3)
memory usage: 1.6+ MB


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных в задании, создайем столбец total_income_category с категориями: 
0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001 - 1000000 'B'
1000001 и выше — 'A'.

In [38]:
def income_group(row):
    income = row['total_income']
    if income <= 30000:
        return 'E'
    if income <= 50000 and income > 30000:
        return 'D'
    if income <= 200000 and income > 50000:
        return 'C'
    if income <= 10000000 and income > 200000:
        return 'B'
    if income > 10000000:
        return 'A'

Data['total_income_category'] = Data.apply(income_group, axis=1)

Data.info()#проверка
Data.head(15) # проверка

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21470 entries, 0 to 21469
Data columns (total 12 columns):
index                    21470 non-null int64
children                 21470 non-null int64
dob_years                21470 non-null int64
education_id             21470 non-null int64
family_status_id         21470 non-null int64
gender                   21470 non-null object
income_type              21470 non-null object
debt                     21470 non-null int64
total_income             21470 non-null int64
purpose                  21470 non-null object
total_income_category    21470 non-null object
purpose_category         21470 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,index,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,0,1,42,0,0,F,сотрудник,0,253876,покупка жилья,B,операции с недвижимостью
1,1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,2,0,33,1,0,M,сотрудник,0,145886,покупка жилья,C,операции с недвижимостью
3,3,3,32,1,0,M,сотрудник,0,267629,дополнительное образование,B,получение образования
4,4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,5,0,27,0,1,M,компаньон,0,255764,покупка жилья,B,операции с недвижимостью
6,6,0,43,0,0,F,компаньон,0,240526,операции с жильем,B,операции с недвижимостью
7,7,0,50,1,0,M,сотрудник,0,135824,образование,C,получение образования
8,8,2,35,0,1,F,сотрудник,0,95857,на проведение свадьбы,C,проведение свадьбы
9,9,0,41,1,0,M,сотрудник,0,144426,покупка жилья для семьи,C,операции с недвижимостью


### Шаг 2.7. Категоризация целей кредита.

 Создание функции, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который 
войдут следующие категории: 'операции с автомобилем', 'операции с недвижимостью', 'проведение свадьбы',
'получение образования'.

In [26]:
def purpose_cat(list):
    if 'автомобил' in list:
        return "операции с автомобилем"
    if "образовани" in list:
        return "получение образования"
    if "свадьб" in list:
        return "проведение свадьбы"
    if "недвижимость" or "жиль" in list:
        return "операции с недвижимостью"
Data['purpose_category'] = Data['purpose'].apply(purpose_cat)

Data.info()#проверка
#Data.head(50) # проверка
print(Data['purpose_category'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21470 entries, 0 to 21469
Data columns (total 12 columns):
index                    21470 non-null int64
children                 21470 non-null int64
dob_years                21470 non-null int64
education_id             21470 non-null int64
family_status_id         21470 non-null int64
gender                   21470 non-null object
income_type              21470 non-null object
debt                     21470 non-null int64
total_income             21470 non-null int64
purpose                  21470 non-null object
total_income_category    21470 non-null object
purpose_category         21470 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
операции с недвижимостью    10813
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64


Расчет Зависимость между количеством детей и возвратом кредита в срок (для ответов на поставленные вопросы и выводов) функция, что посчитает нам отношение в процентах, будем использовать для ответов на все вопросы

In [27]:
def make_proportion(pdSerises):
    return str(round((pdSerises.sum() / pdSerises.count()) * 100, 2)) + '%'

Построим сводную таблицу для ответа на вопрос

In [28]:
children_to_debt_relation = Data.pivot_table(index=['children'], values=["debt"], aggfunc=['sum', 'count', make_proportion])

Сортируем, чтобы сразу видеть у кого ситуация лучше

In [29]:
children_to_debt_relation = children_to_debt_relation.sort_values(by=('make_proportion', 'debt'))
print('Зависимость между количеством детей и возвратом кредита в срок')
print(children_to_debt_relation)

Зависимость между количеством детей и возвратом кредита в срок
           sum  count make_proportion
          debt   debt            debt
children                             
5            0      9            0.0%
0         1064  14153           7.52%
3           27    330           8.18%
1          444   4809           9.23%
2          202   2128           9.49%
4            4     41           9.76%


Вывод  
Если нет детей, то вероятность возникновения задолженности чуть ниже, чем при их наличии. При этом, не зависимо от количества детей, задолженность возникает примерно на том же уровне, существенной разницы в вероятности просрочки не наблюдается. Цифры в отношении семей с пятью детьми не принимаем во внимание по причине не существенного размера выборки.    

Расчет Зависимость между семейным положением и возвратом кредита в срок(для ответов на поставленные вопросы и выводов)
поскольку мы до этого удаляли из основных данных текстовые значения family_status, тут необходимо их вернуть

In [30]:
Data_with_family_status = Data.merge(family_status_dict, on='family_status_id', how='left')

Построим сводную таблицу для ответа на вопрос (по фрейму со значениями из словаря)

In [31]:
family_status_to_debt_relation = Data_with_family_status.pivot_table(index=['family_status'], values=["debt"], aggfunc=['sum', 'count', make_proportion])

Сортируем, чтобы сразу видеть у кого ситуация лучше

In [32]:
family_status_to_debt_relation = family_status_to_debt_relation.sort_values(by=('make_proportion', 'debt'))
print('Зависимость между семейным положением и возвратом кредита в срок')
print(family_status_to_debt_relation)

Зависимость между семейным положением и возвратом кредита в срок
                       sum  count make_proportion
                      debt   debt            debt
family_status                                    
вдовец / вдова          63    959           6.57%
в разводе               85   1195           7.11%
женат / замужем        931  12344           7.54%
гражданский брак       388   4162           9.32%
Не женат / не замужем  274   2810           9.75%


Вывод  
Как следует из проанализированной информации наиболее надежная категория заемщиков вдовец/вдова и люди в разводе. Возможная причина этого - отсутствие иных членов семьи, в связи с чем  кредит берется более осознанно. Самая "ненадежная" категория заемщиков - неженатые и состоящие в гражданском браке.  

Расчет Зависимость между уровнем дохода и возвратом кредита в срок ((для ответов на поставленные вопросы и выводов)построим сводную таблицу для ответа на вопрос

In [33]:
total_income_to_debt_relation = Data.pivot_table(index=['total_income_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])

Сортируем, чтобы сразу видеть у кого ситуация лучше

In [34]:
total_income_to_debt_relation = total_income_to_debt_relation.sort_values(by=('make_proportion', 'debt'))
print('Зависимость между уровнем дохода и возвратом кредита в срок')
print(total_income_to_debt_relation)

Зависимость между уровнем дохода и возвратом кредита в срок
                        sum  count make_proportion
                       debt   debt            debt
total_income_category                             
D                        21    350            6.0%
B                       366   5248           6.97%
C                      1352  15850           8.53%
E                         2     22           9.09%


Вывод  
Из полученной информации очевидно, что меньше всего просрочка у заемщиков с доходом в диапазоне (от 30 001 до 50 000 руб), при этом у самых малообеспеченных (от 0 до 30 000) процент просрочки максимальный. Также "опасной" группой заемщиков с доходами 500001 - 200000 - у условного "зарабатывающего и потребляющего" среднего класса. 


Расчет Зависимость между целями кредита и его возврата в срок (для ответов на поставленные вопросы и выводов) построим сводную таблицу для ответа на вопрос

In [35]:
purpose_category_to_debt_relation = Data.pivot_table(index=['purpose_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])

Сортируем, чтобы сразу видеть у кого ситуация лучше

In [36]:
purpose_category_to_debt_relation = purpose_category_to_debt_relation.sort_values(by=('make_proportion', 'debt'))
print('Зависимость между целями кредита и его возврата в срок') 
print(purpose_category_to_debt_relation)

Зависимость между целями кредита и его возврата в срок
                          sum  count make_proportion
                         debt   debt            debt
purpose_category                                    
операции с недвижимостью  782  10813           7.23%
проведение свадьбы        186   2335           7.97%
получение образования     370   4014           9.22%
операции с автомобилем    403   4308           9.35%


Вывод  
Из полученной информации очевидно, что выгоднее всего давать кредиты на недвижимость, а автомобиль и образование - в зоне риска. Хоть абсолютная разница между соответствующими показателями относительно друг друга не такая большая.  

##### Вопрос 1:

•	Есть ли зависимость между количеством детей и возвратом кредита в срок? 
•	Есть ли зависимость между семейным положением и возвратом кредита в срок? 
•	Есть ли зависимость между уровнем дохода и возвратом кредита в срок? 
•	Как разные цели кредита влияют на его возврат в срок? 


##### Вывод 1:

Да, указанные зависимости наблюдаются, пояснения по ним указаны непосредственно после каждой расчетной таблицей.








## Общий вывод:

Проведенный анализ показывает, что самый лучший заемщик - это: 
    - человек без детей; 
    - состоявший ранее в официальных отношениях; 
    - имеющий уровень дохода ниже среднего или средний; 
    - берущий кредит на недвижимость. 
"Разброс" просроченных выплат по каждой из категорий респондентов примерно одинаковый. В среднем, это диапазоны от 7 до 9 процентов. В остальном разница в абсолютных цифрах не такая большая. Однако в относительных разница составляет в среднем 2 процента, что означает разницу в вероятности просрочки платежа на 20% между более и менее надежными заемщиками 

Более детально по категориям: 
Влияние количества детей на вероятность просрочки платежа 
5 детей 0.0% (не принимаем в расчет) 
0 детей 7.53% 
3 ребенка 8.18% 
1 ребенок 9.23% 
2 ребенка 9.49% 
4 ребенка 9.76% 
Абсолютная разница между минимальным и максимальным значением - 2.23% 
Относительная разница между минимальным и максимальным значением - 22.84% 

Влияние семейного положения на вероятность просрочки платежа 
вдовец / вдова         6.57% 
в разводе              7.11% 
женат / замужем        7.55% 
гражданский брак       9.35% 
Не женат / не замужем  9.75% 
Абсолютная разница между минимальным и максимальным значением - 3.18% 
Относительная разница между минимальным и максимальным значением - 32.61% 

Влияние уровня дохода на просрочку платежа 
D 30001–50000    6.0% 
B 200001-1000000 7.07% 
C 50001–200000   8.49% 
E 0–30000        9.09% 
Абсолютная разница между минимальным и максимальным значением - 3.09% 
Относительная разница между минимальным и максимальным значением - 33.99% 

Влияние цели кредита на вероятность возврата кредита в срок 
операции с недвижимостью  7.23% 
проведение свадьбы        8.0% 
получение образования     9.22% 
операции с автомобилем    9.36% 
Абсолютная разница между минимальным и максимальным значением - 2.13% 
Относительная разница между минимальным и максимальным значением - 22.75% 

Таким образом, несмотря на схожий в процентном отношении разброс значений, вероятность просрочки зависит от (по убыванию): 
    - уровеня дохода 
    - семейного положения 
    - количество детей 
    - цель кредитования.  
Условно "идеальный заемщик" - это респондент с относительно не большимим (но не минимальными) доходами, разведенный или вдова/вдовец. 
 